In [30]:
import time
import random
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

In [33]:
train = pd.read_csv('./datasets/train.csv')
test  = pd.read_csv('./datasets/test.csv')
submission = pd.read_csv('./datasets/submission.csv')

In [3]:
train.head(25)

,id,digit,letter,0,1,2,3,4,5,6,...,774,775,776,777,778,779,780,781,782,783
0,1,5,L,1,1,1,4,3,0,0,...,2,1,0,1,2,4,4,4,3,4
1,2,0,B,0,4,0,0,4,1,1,...,0,3,0,1,4,1,4,2,1,2
2,3,4,L,1,1,2,2,1,1,1,...,3,3,3,0,2,0,3,0,2,2
3,4,9,D,1,2,0,2,0,4,0,...,3,3,2,0,1,4,0,0,1,1
4,5,6,A,3,0,2,4,0,3,0,...,4,4,3,2,1,3,4,3,1,2
5,6,8,C,4,3,0,3,3,4,3,...,4,3,0,4,4,4,2,2,3,4
6,7,1,Q,0,0,4,2,4,0,4,...,4,3,2,0,4,4,4,3,1,3
7,8,3,M,1,0,3,4,4,0,2,...,2,0,4,4,4,0,2,2,3,1
8,9,6,F,0,1,0,4,0,1,2,...,3,2,4,4,4,1,0,1,3,3
9,10,8,J,4,3,4,0,0,0,4,...,2,0,0,1,3,0,3,3,1,2


In [36]:
data_dir = './datasets/train.csv'
data = pd.read_csv(data_dir)

del data['id']
del data['letter']

label = data['digit']
input = []

for i in range(2048):
    temp = data.iloc[i,1:].values.reshape(28, 28,1)
    input.append(temp)

# list 자료형으로는 img를 나타낼 수 없으므로 numpy 자료형으로 바꾸어 준다.    
# 데이터 타입 에러 발생으로 np.float32를 추가해준다
input = (np.array(input)).astype(np.float32)
label = (np.array(label)).astype(np.float32)

if input.ndim == 3:
    input = input[:, :, :, np.newaxis]

input = np.moveaxis(input[:, :, :, :], -1, 1)

#input.shape

(2048, 1, 28, 28)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(input, label, test_size=0.05, random_state=7777)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1945, 1, 28, 28), (1945,), (103, 1, 28, 28), (103,))

In [47]:
train_input, valid_input, train_label,valid_label = train_test_split(input, label, test_size=0.05, random_state=7)

dataset_train = {'label': torch.from_numpy(train_label), 'input': torch.from_numpy(train_input)}
dataset_valid = {'label': valid_label, 'input': valid_input}

len(dataset_train)

2